In [ ]:
#!pip install numpy==1.26.4
#!pip install zarr==2.18.7

# Multislice simulations with *ab*TEM - basics

This tutorial is a short introduction to image simulation with *ab*TEM. The tutorial covers some basic principles such as potential and wave functions, and presents examples for CBED, STEM and HRTEM and simulations. For more in-depth information, see the following resources:

* [The *ab*TEM documentation](https://abtem.github.io/doc/intro.html)
* [The *ab*TEM walkthrough](https://abtem.github.io/doc/user_guide/walkthrough/walkthrough.html)

We have also contributed to a computational article "[A Practical Guide to Scanning and Transmission Electron Microscopy Simulations](https://www.elementalmicroscopy.com/articles/EM000005/)", which gives an in-depth introduction into TEM simulations using *ab*TEM.

### Contents:

1. <a href='#import'> Import atomic model
2. <a href='#potentials'> Potentials with the independent atom model
3. <a href='#probes'> Probe wave functions
5. <a href='#multislice'> Multislice simulation with a probe
6. <a href='#scan'> Scanned multislice simulation

In [ ]:
%matplotlib inline

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np

abtem.config.set({"visualize.cmap": "viridis"})
abtem.config.set({"visualize.continuous_update": True})
abtem.config.set({"visualize.autoscale": True})
#abtem.config.set({"visualize.reciprocal_space_units": "mrad"})
abtem.config.set({"device": "cpu"})
abtem.config.set({"fft": "fftw"});

## Import atomic model <a id='import'></a>

To start running image simulations, we need an atomic model. Creating an atomic model was covered in the previous tutorial, so if you do not have the file "sto_lto.cif", please run that notebook first.

In [ ]:
atoms = ase.io.read("sto_lto.cif")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
abtem.show_atoms(atoms, ax=ax1, plane="xy", title="Beam view")
abtem.show_atoms(atoms, ax=ax2, plane="yz", title="Side view", legend=True);

## Creating a `Potential` with the independent atom model <a id='potentials'></a>
We use the indepedent atom model (IAM) to create the electrostatic potential of the sample. In this standard approximation, the potential is a superposition of parameterized isolated atomic potentials. More information on potentials in *ab*TEM may be found in our [walkthrough](https://abtem.github.io/doc/user_guide/walkthrough/potentials.html).

Our default parametrization is the one by Lobato, though also Kirkland and Peng are available. For most practical purposes, the differences are minor.

To define a `Potential`, we need to provide an ASE `Atoms` object, a sampling rate (or pixel size) in $x$ and $y$, and a slice thickness in the $z$-direction (the propagation direction).

The multislice algorithm is only accurate in the limit of good (small) sampling rate and thin slices, but improving these parameters also increases computational cost. A sensible value for the sampling is between $\mathrm{0.05} \ \mathrm{Å}$ and $0.02 \ \mathrm{Å}$, and the slice thickness is typically between $1.0 \ \mathrm{Å}$ and $0.025 \ \mathrm{Å}$, though the value of $2.0 \ \mathrm{Å}$ used here may suffice.

Both should be treated as convergence parameters, which need to be small enough so that the simulated signal does not depend on the exact values. Some more information on convergence can be found in an [appendix](https://abtem.github.io/doc/user_guide/appendix/convergence.html) of our online documentation.

In [ ]:
potential = abtem.Potential(atoms, sampling=0.05, slice_thickness=2)

The potential has 32 slices along the $z$ propagation direction, as may be determined from getting its length.

In [ ]:
len(potential)

This is because the cell is atomic model is 63.8 $\mathrm{Å}$ in height, which can be divided into approximately that many 2 $\mathrm{Å}$ slices.

In [ ]:
atoms.cell[2,2] / 2

*ab*TEM will automatically adapt the requested slice thickness (and sampling) to result in an integer number.

In [ ]:
potential.slice_thickness[0]

That length is the first axis of the potential corresponding to the propagation direction, followed by $x$ and $y$, giving the full shape.

In [ ]:
potential.shape

Again, the $x$ and $y$ sizes are given by the size of the simulation cell divided into pixels whose size is given by the sampling of 0.05 $\mathrm{Å}$.

In [ ]:
np.diag(atoms.cell[:2])/0.05

In [ ]:
potential

The `.build` method is available for many *ab*TEM simulation objects. This method will convert them into a static array-based object.

In [ ]:
potential_array = potential.build()

*ab*TEM is evaluated lazily using the Dask library, which means that the resulting array is not computed immediately – we have only created the instructions for computing the array. 

In [ ]:
potential_array.array

To actually calculate the array, we run the computation using the `.compute` method.

In [ ]:
potential_array.compute()

After computing, the built potential is described as a `NumPy` array. The beginning and end of the array corresponds to the vacuum regions at the edges of the cell, and thus the potential is zero there.

In [ ]:
potential_array.array

We can show the potential using the `.show` method - to display a 2D image, it is by default projected to the $xy$ plane.

In [ ]:
potential_array.show(cbar=True);

*ab*TEM has some features for showing the simulation objects interactively. Here, we convert the potential slice to a stack of images and show the result with `interact=True`, allowing us to scroll through slices. 

__Note__: This requires that the Matplotlib backend is using [`ipympl`](https://matplotlib.org/ipympl/); we'll revert back to inline plotting for regular graphics.

In [ ]:
%matplotlib ipympl
potential_array.to_images().show(interact=True, cbar=True);

## Multislice simulations

Above, we described how to integrate the potential into a series of thin slices $V_n$ along the $z$-axis. Now we will describe how the potential slices are used in the multislice algorithm.

Given a wave function of fast electrons at the entrance of the $n$'th slice, $\psi_n(\vec{r})$, and a weak potential slice $V_n$, the wave function at exit plane of that slice may be written as

$$
    \psi_{n + 1}(\vec{r}) = p(\vec{r}) * \left[t_n(\vec{r}) \psi_n(\vec{r}) \right]
$$

where 

$$
    p(\vec{r}) = \frac{1}{i \lambda \Delta z}\exp\left[\frac{i\pi}{\lambda \Delta z} r^2 \right]
$$

is the [Fresnel free-space operator](https://en.wikipedia.org/wiki/Fresnel_diffraction) for propagation by a distance $\Delta z$ along the $z$-axis, $*$ is the convolution operator and 

$$
    t_n(\vec{r}) = \exp\left[i\sigma V_n(\vec{r})\right] 
$$

is the transmission function that applies a phase shift proportional to the magnitude of the potential slice $V_n$, where the proportionality constant, $\sigma$, is called the interaction constant. The derivation of these equations may be found in textbooks or in "[A Practical Guide to Scanning and Transmission Electron Microscopy Simulations](https://www.elementalmicroscopy.com/articles/EM000005/)".

Using the [Fourier convolution theorem](https://en.wikipedia.org/wiki/Convolution_theorem), we can write the multislice equation as

$$
    \psi_{n+1}(\vec{r}) = \mathcal{F}^{-1} \left\{P(\vec{k}) \ \mathcal{F}\left[t(\vec{r}) \psi_n(\vec{r})\right] \right\}  := \mathcal{M}_n \psi_n(\vec{r}) \quad ,
$$

where 

$$
    P(\vec{k}) = \exp(-i \pi \lambda k^2 \Delta z)
$$

is the Fresnel free space propagator in reciprocal space and $\mathcal{F}$ and $\mathcal{F}^{-1}$ is the Fourier transform and its inverse. For brevity, we have defined a multislice operator, $\mathcal{M}_n$, acting on a wave function to step it forward through the $n$'th potential slice. 

Thus, given an initial wave function $\psi_0$, we can obtain the exit wave function for a potential with slice indices $n=1,\ldots N$ by sequentially applying these operators

$$
    \psi_{exit}(\vec{r}) := \psi_{N}(\vec{r}) = \mathcal{M}_N \mathcal{M}_{N-1} \ldots \mathcal{M}_{1} \psi_0(\vec{r}) \quad .
$$

## Multislice simulations with plane waves

Below we create a `Potential` representing gold with a lattice constant of $4.08 \ \mathrm{Å}$ in the $\left<100\right>$ zone axis and use a `PlaneWave` as the initial wave function.

In [ ]:
plt.close()
%matplotlib inline

unit_cell = ase.build.bulk("Au", cubic=True)

gold = unit_cell * (1, 1, 30)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
abtem.show_atoms(gold, ax=ax1, title="Beam view")
abtem.show_atoms(gold, ax=ax2, plane="xz", title="Side view", linewidth=0);

After repeating the structure along $z$, the simulation cell has a size of $(4.08\times4.08\times122.4) \ \mathrm{Å}$.

In [ ]:
gold.cell

We set the slice thickness to half the unit cell height $4.08 / 2 = 2.04 \ \mathrm{Å}$ and the $xy$-sampling of the potential to $0.04 \ \mathrm{Å}$.

```{tip} Slice thickness
It is beneficial, but not always possible, to make the slice thickness the same as the distance between the atomic planes of the crystal. Doing so typically allows you to use thicker slices.

In all cases, the multislice algorithm is only accurate in the limit of small slice thicknesses, and you should always make sure that your calculation is sufficiently converged with respect to the slice thickness.  
```

We then create a `PlaneWave` of 200 keV to describe our parallel probe.

In [ ]:
pot_gold = abtem.Potential(gold, slice_thickness=4.08 / 2, sampling=0.04)

plane_wave = abtem.PlaneWave(energy=200e3)

We set up our calculation by calling the `multislice` method, producing an exit wave function.

In [ ]:
exit_wave = plane_wave.multislice(pot_gold)
exit_wave.shape

As discussed previously, nothing is yet calculated. To execute the simulation we need to call `compute`.

```{note}
Notice that we did not provide the sampling or extent of the wave function above, and so the wave function automatically adopted the grid of the potential. A `GridUndefinedError` will be thrown if the grid is not defined for _both_ the wave function and potential.
```

In [ ]:
exit_wave.compute();

We can then visualize the intensity of the resulting exit wave function.

In [ ]:
exit_wave.intensity().show(common_color_scale=True, cbar=True);

In realistic HRTEM experiments, the wave functions have to be magnified by an objective lens which introduces aberrations and effectively cuts off large scattering angles.

Here, we apply a defocus of $-50 \ \mathrm{Å}$ and an objective aperture of $20 \ \mathrm{mrad}$.

In [ ]:
exit_wave.apply_ctf(defocus=-50, semiangle_cutoff=20).intensity().show(cbar=True);

Aberrations of all kinds can be defined as shown in our [documentation on the contrast transfer function](contrast_transfer_function.ipynb). Relatively sophisticated models of [partial coherence](https://abtem.github.io/doc/user_guide/tutorials/partial_coherence.html) are also supported.

### Electron diffraction patterns

Instead of an image, we can instead simulate a [selected area diffraction (SAD)](https://en.wikipedia.org/wiki/Selected_area_diffraction) experiment by using the `DiffractionPatterns` method. We use `block_direct=True` to block the direct beam: it typically has a much higher intensity than the scattered beams, and thus it is typically not possible to show it on the same scale.

In [ ]:
diffraction_patterns = exit_wave.diffraction_patterns()

diffraction_patterns.block_direct().show(units="mrad", cbar=True)

You may wonder; why do the diffraction spots look like squares? This is because the incoming wave function is a periodic and infinite plane wave, hence the intersection with the [Ewald sphere](https://en.wikipedia.org/wiki/Ewald%27s_sphere) is pointlike. However, since we are discretizing the wave function on a square grid (i.e. pixels), the spots can only be as small as single pixels. In real SAD experiments, the spot size is broadened due to the finite extent of the crystal as well instrumental effects.

We can use the `index_diffraction_spots` method to create a represention of SAD patterns as a mapping of Miller indices to the intensity of the corresponding reflections. The *conventional* unit cell have to be provided in order to index the pattern, we can provide this as the _unit cell_ of the gold crystal we created earlier, we cannot use the the repeated cell.

In [ ]:
indexed_spots = diffraction_patterns.crop(120).index_diffraction_spots(cell=unit_cell)

indexed_spots

The `IndexedDiffractionPatterns` facilitates the creation of a visualization that corresponds closer with typical textbook illustrations, where the area of the disks are proportional to the intensity. The disks may be scaled using `scale` and to limit cluttering miller index annotations maybe excluded by providing a `threshold` in the `annotation_kwargs` argument. 

In [ ]:
visualization = (
    indexed_spots.block_direct()
    .show(
        scale=4,
        cbar=True,
        annotation_kwargs={"threshold": 0.003, "fontsize": 6},
        figsize=(8, 8),
        units="mrad"
    )
)

We see that the $\{100\}$ reflections are extinguished, as is expected from the selection rules of an F-centered crystal. We can also observe that the $\left<220\right>$ spots have significantly higher intensity than the $\left<200\right>$ spots; this is due to dynamical scattering — which is accounted for by the multislice algorithm.

It is possible to obtain a [pandas](https://pandas.pydata.org/) dataframe of the intensity values of the indexed diffraction spots. We only include spots with an intensity of at least $0.005$ (as a fraction of the incoming beam) using the `remove_low_intensity` method.

In [ ]:
df = indexed_spots.remove_low_intensity(0.005).to_dataframe()
df

## Writing exit wave functions

You can write exit wave function directly to disk, which will also trigger the computation to run.

In [ ]:
exit_wave = plane_wave.multislice(potential)

In [ ]:
exit_wave.to_zarr("./data/exit_waves.zarr", overwrite=True);

We can read the wave function back in as shown below, and see that they are identical to the calculated exit wave.

```{tip}
By writing a calculation directly to disk, it is performed in chunks and thus will require far less total memory than when using the `compute()` method directly.
```

In [ ]:
imported_wave = abtem.from_zarr("./data/exit_waves.zarr").compute()

assert imported_wave.compute() == exit_wave.compute()

## Thickness series

*ab*TEM easily allows us to obtain the wave function at intermediate steps of the multislice algorithm, thus allowing us to see how the wave function evolves as it passes through the potential. To create such a *thickness series* we set the `exit_planes` keyword of the *potential*. `exit_planes` may be given as a tuple of slice indices at which to return the wave function, or simply as a single integer to indicate the step between those slice indices.

Below we create a `Potential` as above, but we set `exit_planes=4`. When running the multislice simulation we obtain an *ensemble* of wave functions $\psi_n(\vec{r})$ at slice indices $n=0,2,4,\ldots$, ie. after every unit cell.

In [ ]:
potential_series = abtem.Potential(
    gold, slice_thickness=4.08 / 2, sampling=0.05, exit_planes=2
)

exit_wave_series = plane_wave.multislice(potential_series).compute()

The thickness series consists of $31$ exit waves.

In [ ]:
exit_wave_series.shape

We do not want to show all $31$ exit waves, hence, we can select every fifth exit wave using standard NumPy indexing. We show the wave function intensity as an exploded plot, enforcing a joint intensity color scale with a given maximum value.

In [ ]:
exit_wave_series[5::5].show(
    explode=True,
    figsize=(14, 5),
    common_color_scale=True,
    cbar=True,
    vmin=0,
    vmax=6,
);

We can naturally get the indexed diffraction patterns for the series.

In [ ]:
diffraction_patterns_series = exit_wave_series.diffraction_patterns(max_angle=120)

indexed_spots = diffraction_patterns_series.index_diffraction_spots(
    cell=4.08,
)

For a relatively heavy atom such as gold, the diffraction spots in the first order Laue zone are excited already at $20 \ \mathrm{Å}$. 


In [ ]:
indexed_spots[5::5].block_direct().show(
    explode=True,
    common_color_scale=True,
    figsize=(14, 5),
    cbar=True,
    scale=0.3,
    annotation_kwargs={"threshold": 1.0, "fontsize": 7},
);

The diffraction intensities can be obtained as a [pandas](https://pandas.pydata.org/) dataframe indexed by the thickness. We set a threshold to only include spots which *at some thickness* have an intensity above $0.005$.


In [ ]:
df = indexed_spots.remove_low_intensity(0.005).to_dataframe()

We select every sixth diffraction spot and show the dataframe.


In [ ]:
df.iloc[::6]

We can now leverage all the pandas features, for example, we can select three diffraction spots and plot their intensities together. 

In [ ]:
ax = df[["0 4 0", "0 2 0", "2 2 0"]].plot()
ax.set_ylabel("intensity [arb. unit]");

## Creating a focused `Probe` wave functions <a id='probes'></a>

The multislice algorithm works by propagating the $xy$ part a the wave function through the electrostatic potential along the $z$-axis. In STEM, the wave function is a focused beam of electrons. The convention used in *ab*TEM is a probe defined by

$$
    \phi(\mathbf{k}, \mathbf{r}_0) = A(k) \exp(-i \chi(\mathbf{k})) \exp(-i 2 \pi \mathbf{k} \cdot \mathbf{r}_p) \quad ,
$$

where $\mathbf{k} = (k_x, k_y)$ is the spatial frequency, $A(k)$ is the condenser aperture function and $\chi(\mathbf{k})$ is the phase error, and $\mathbf{r}_p = (x_p, y_p)$ is the probe position. (See our [walkthrough](https://abtem.github.io/doc/user_guide/walkthrough/contrast_transfer_function.html) on wave functions for more detail.)

If the microscope is well aligned, off-axis aberrations are small and the phase error is dominated by defocus and spherical aberration

$$
    \chi(k) \approx \frac{2\pi}{\lambda}\left( \frac{\lambda^2 k^2}{2} \Delta f + \frac{\lambda^4 k^4}{4} C_s \right) \quad ,
$$

where $\Delta f$ is the defocus and $C_s$ is the third order spherical aberration. (See our [walkthrough](https://abtem.github.io/doc/user_guide/walkthrough/contrast_transfer_function.html) on the contrast transfer function for more.)

We create a probe with an energy of $150 \ \mathrm{keV}$, a defocus of $50 \ \mathrm{Å}$, and a convergence semiangle of $20 \ \mathrm{mrad}$.

__Note__: Positive defocus is equivalent to backward free-space propagation, i.e. a probe with positive defocus is "in focus" inside the sample. Please see our [Appendix on conventions](https://abtem.github.io/doc/user_guide/appendix/conventions.html) for more detail.

In [ ]:
probe = abtem.Probe(energy=150e3, defocus=50, semiangle_cutoff=20, Cs=0.0)

We did not specify any grid for our probe, but we can simply match it to our potential.

In [ ]:
probe.grid.match(potential)

We can `.build` the probe to obtain an (lazy) array representation, and then compute it.

In [ ]:
probe_waves = probe.build()

probe_waves.array

In [ ]:
probe_waves.compute();

The wave function intensity can be shown in real or reciprocal space using the `.intensity` or `.diffraction_patterns` methods. 

__Note__: We used `grid.match`, hence the probe grid matches the potential. 

In [ ]:
probe_waves.intensity().show(title="Probe intensity");

We can create probes with multiple different values of defocus (or other parameters) by providing a sequence of values, e.g. as a `NumPy` array. Here we create an ensemble of $5$ probes with a defocus ranging from $0$ to $200 \ \mathrm{Å}$, and a further $5$ semiangle cutoff values from $10$ to $30$ mrad.

In [ ]:
np.linspace(10, 30, 5)

In [ ]:
focal_series = np.linspace(0, 200, 5)
semiangle_series = np.linspace(10, 30, 5)

focal_series_probe = abtem.Probe(
    energy=200e3, defocus=focal_series, semiangle_cutoff=semiangle_series, extent=10, sampling=0.05
)

We make an interactive visualization to scroll through the probe ensemble. 

__Note__: The probe `extent` is too small to represent a probe with the largest defocus in the series. We are seeing errors due to probe self-interaction causing the probe to have only four-fold rotational symmetry as a consequence of the square probe window.

In [ ]:
%matplotlib ipympl
focal_series_probe.build().compute().show(interact=True, cbar=True);

As an alternative to the interactive plots, we can set `explode=True` to show the entire ensemble. Note how *ab*TEM automatically creates and labels visualizations based on the parameter ensembles.

In [ ]:
plt.close()
%matplotlib inline
plt.close() # Required to get a correctly displaced plot after switching from ipympl to inline.

focal_series_probe.build().show(explode=True, figsize=(12, 12));

In [ ]:
probewaves = focal_series_probe.build()

Typically one would simply use one value of defocus and semiangle for a single simulation, but the ensembles can be very powerful for exploring the effects of parameters on the results, and for easy visualization.

## Multislice simulation with a `Probe` (CBED) <a id='multislice'></a>
We use the multislice algorithm to propagate the probe through the potential. We can choose where to place the probe by setting the `scan` argument. Here we place the probe at $\mathbf{r}_p = (8,8) \ \mathrm{Å}$.

In [ ]:
position = (8, 8)

exit_wave = focal_series_probe.multislice(potential, scan=position)

exit_wave.array

We run `.compute` to calculate the exit wave.

In [ ]:
exit_wave.compute()

We can show the exit wave intensity.

In [ ]:
exit_wave_image = exit_wave.intensity()

exit_wave_image.show();

We can use an interactive visualization to show the diffraction on different power scales; reducing the power will reveal the weaker diffraction outside the bright-field disk.

In [ ]:
%matplotlib ipympl

exit_wave_diffraction = exit_wave.diffraction_patterns(max_angle="full")

exit_wave_diffraction.show(cbar=True, units="mrad", power=0.5, interact=True);

We set `max_angle="full"` above to show the diffraction pattern up to the cutoff value, but the maximum available simulated angle does not correspond to a full simulation grid. Instead, real-space sampling determines the maximum simulated scattering angle. 

The sampling defines the maximum spatial frequency $k_{max}$ via

$$ k_{max} = \frac{1}{2d} \quad , $$

where $d$ is the real-space sampling distance. To counteract aliasing artifacts due to the periodicity assumption of a discrete Fourier transform, *ab*TEM supresses spatial frequencies above 2 / 3 of the maximum scattering angle, further reducing the maximum effective scattering angle by that factor. Hence, the maximum scattering angle $\alpha_{max}$ is given by

$$ \alpha_{max} = \frac{2}{3}\frac{\lambda}{2p} \quad , $$

where $\lambda$ is the relativistic electron wavelength. 

## Scanned multislice simulation <a id='scan'></a>

Scanning imaging modes such as STEM works by rastering an electron probe across a sample pixel by pixel and recording the scattering signal. 

We create a grid scan and set the sampling (probe step size) to the Nyquist sampling of the probe. The resulting image can be interpolated to the typically much higher experimental sampling rate.

We only scan across $1 / 3$ of the potential along $x$ because it is repeated three times in this direction. This is most conviniently done using fractional coordinates, which then requires us to provide the potential as well.

In [ ]:
scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1),
    sampling=probe.ctf.nyquist_sampling,
    fractional=True,
    potential=potential,
)

We can overlay the scan region over the atoms to confirm it matches what we desired.

In [ ]:
plt.close()
%matplotlib inline
plt.close()

fig, ax = abtem.show_atoms(atoms)
scan.add_to_plot(ax);

__Note__: The scan `sampling` should not be confused with the wave function `sampling` due to discretization. The former is equivalent to the probe step size, while the second has no experimental equivalent.

In *ab*TEM the exit waves are "detected" using a detector object. There are several different types of detectors, and the most basic one, the `AnnularDetector`, may be used for bright-field, medium- or high-angle annular dark-field microscopy, depending on the angular integration range.

The integration region is given by an inner and an outer radius in mrad. Below, we create three different annular detectors.

In [ ]:
bright = abtem.AnnularDetector(inner=0, outer=20)
maadf = abtem.AnnularDetector(inner=50, outer=120)
haadf = abtem.AnnularDetector(inner=100, outer=180)

detectors = [bright, maadf, haadf]

The outer radius can only be as large as the maximum simulated scattering angle.

In [ ]:
print(f"alpha_max = {min(probe.cutoff_angles):.1f} mrad")

The detector regions, given a wave function, may be retrieved using the get detector region method.

In [ ]:
bright_region = bright.get_detector_region(probe)
maadf_region = maadf.get_detector_region(probe)
haadf_region = haadf.get_detector_region(probe)

To conveniently show all the regions together we stack them, providing a name for each, and show them with `explode=True`.

In [ ]:
stacked_regions = abtem.stack(
    (bright_region, maadf_region, haadf_region), ("bright", "MAADF", "HAADF")
)

visualization = stacked_regions.show(explode=True, units="mrad", figsize=(8, 4))

The scanned multislice simulations are performed as shown below and will run in a minute or two on most laptops.

In [ ]:
scanned_measurements = probe.scan(
    scan=scan,
    detectors=detectors,
    potential=potential,
);

scanned_measurements.compute();

In [ ]:
scan.shape

The output is given as a list of three `Images` objects, one for each detector. We can stack show the measurements with `explode=True`.

In [ ]:
stacked_measurements = abtem.stack(scanned_measurements, ("BF", "MAADF", "HAADF"))

stacked_measurements.show(explode=True);

## Post-processing STEM measurements

STEM simulations usually requires some post-processing, so we apply the most common steps below.

### Interpolation

We saved a lot of computational time by scanning at the Nyquist frequency, but the result is quite pixelated. To address this, we interpolate the images to a sampling of $0.1 \ \mathrm{Å / pixel}$. *ab*TEM's default interpolation algorithm is Fourier-space padding (but spline interpolation is also available, which is more appropriate if the image in non-periodic).

In [ ]:
interpolated_measurements = stacked_measurements.interpolate(0.1)

interpolated_measurements.show(explode=True);

### Blurring

A finite Gaussian-shaped source will result in a blurring of the image. Vibrations and other instabilities may further contribute to the blur. We apply a Gaussian blur with a standard deviation of $0.5 \ \mathrm{Å}$ (corresponding to a source of approximately that size).

__Note__: We are not including partial temporal incoherence here. See our [tutorial on partial coherence](../tutorials/partial_coherence.ipynb).

In [ ]:
blurred_measurements = interpolated_measurements.gaussian_filter(0.35)

blurred_measurements.show(explode=True);

### Noise

Simulations such as the above corresponds to the limit of infinite electron dose. We can emulate finite dose by drawing random numbers from a Poisson distribution for every pixel.

Before applying the noise, we tile the images to get better statistics; note that to not repeat the same noise, tiling needs to happen first!

In [ ]:
tiled_measurements = blurred_measurements.tile((8, 3))

We apply Poisson noise corresponding a dose per area of $10^4 \ \mathrm{e}^- / \mathrm{Å}^2$.

In [ ]:
tiled_measurements.show(explode=True, figsize=(12, 4), cbar=True);

In [ ]:
noisy_measurements = tiled_measurements.poisson_noise(dose_per_area=1e4, seed=100)

noisy_measurements.show(explode=True, figsize=(12, 4), cbar=True);

# Multislice simulations with *ab*TEM - advanced

This tutorial is a continuation of a short introduction to image simulation with *ab*TEM, covering somewhat more advanced topics.

### Contents:

1. <a href='#4d_stem'> 4D-STEM
2. <a href='#frozen_phonons'> The frozen phonon model
3. <a href='#parallel'> Parallelization

In [ ]:
%matplotlib inline

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np

abtem.config.set({"visualize.cmap": "viridis"})
abtem.config.set({"visualize.continuous_update": True})
abtem.config.set({"visualize.autoscale": True})
# abtem.config.set({"visualize.reciprocal_space_units": "mrad"})
abtem.config.set({"device": "cpu"})
abtem.config.set({"fft": "fftw"});

## 4D-STEM <a id='4d_stem'></a>

To run a 4D-STEM simulation, we only need to change the `AnnularDetector` to a `PixelatedDetector`, which will by default detect the diffraction patterns up the angle corresponding to the largest rectangle inside the antialiasing limit; however, we can choose another maximum angle by setting the `max_angle` argument.

Here we set the maximum angle to $100 \ \mathrm{mrad}$.

In [ ]:
detector = abtem.PixelatedDetector(max_angle=100)

We recreate some the simulation objects from the basic tutorial. 

In [ ]:
atoms = ase.io.read("sto_lto.cif")
abtem.show_atoms(atoms);

Let's say we needed to do 1024$\times$1024 probe positions. How big would the resulting measurement be?

We build a scanned multislice simulation, resulting in a 4D array with $1024\times 1024$ probe positions each producing a $116\times 107$ diffraction pattern.

In [ ]:
potential = abtem.Potential(atoms, sampling=0.1, slice_thickness=2)

probe = abtem.Probe(energy=150e3, defocus=50, semiangle_cutoff=20)

scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1),
    gpts=(1024,1024),
    fractional=True,
    potential=potential,
)

measurement_4d = probe.scan(scan=scan, potential=potential, detectors=detector)

In [ ]:
measurement_4d.array

The size of the dataset is quite large, and it would not fit in my laptop memory. However, it is often preferable to write the simulation results directly to disk instead of storing all in memory. *ab*TEM uses Dask and [zarr](https://zarr.readthedocs.io/en/stable/) to efficiently read and write in manageable chunks; in this case, 7396 chunks across the scan dimensions.

However, it would still be possible to run this calculation by directly writing it to disk, which triggers a computation.

But let's use Nyquist sampling instead to make this example run on laptops.

In [ ]:
scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1),
    #gpts=(1024,1024),
    sampling=probe.ctf.nyquist_sampling,
    fractional=True,
    potential=potential,
)

measurement_4d = probe.scan(scan=scan, potential=potential, detectors=detector)

In [ ]:
measurement_4d.to_zarr("measurement_4d.zarr");

We use the `from_zarr` function to lazily read back the results – note that this way the array is not stored in memory, just a reference to the data on disk!

For this few scan positions, there is no big difference, but this would allow you to analyze much bigger datasets with limited memory.

In [ ]:
measurement_4d.array

In [ ]:
measurement_4d = abtem.from_zarr("measurement_4d.zarr")
measurement_4d.array

To read the entire measurement from disk into memory, we could run `.compute`, but this is often unecessary as most *ab*TEM features works with lazy measurements.

We can index the dataset to retrieve a specific diffraction pattern.

In [ ]:
measurement_4d[1, 1].show(cbar=True);

We can also show the diffraction patterns using an interactive visualization reading each chunk directly from disk.

In [ ]:
%matplotlib ipympl
measurement_4d.show(interact=True, power=0.5, cbar=True);

Since we have the full diffraction pattern, we can integrate between any two scattering angles within the maximum detected angle.

To do this, we first need to bin the diffraction patterns radially. We specify 100 radial bins and 1 azimuthal bin.

In [ ]:
polar_binned = measurement_4d.polar_binning(nbins_radial=100, nbins_azimuthal=1)

polar_binned.array

The `to_image_ensemble` creates a representation of the polar binned diffraction patterns for displaying interactively. We interpolate and tile to get a better visualization.

In [ ]:
binned_images = polar_binned.to_image_ensemble().compute().interpolate(0.1).tile((3, 1))

binned_images.show(interact=True);

In 4D-STEM, some algorithm is typically used for reducing the dataset to 2D. *ab*TEM includes some basic tools for reduction of 4D-STEM data, but for more advanced algorithms you may want to try a package dedicated to 4D-STEM (such as [py4DSTEM](https://github.com/py4dstem/py4DSTEM)).

The center of mass, $\vec{I}_{com}(\vec{r}_p)$, of the diffraction pattern at a probe position, $\vec{r}_p$, may be calculated as

$$
    \vec{I}_{com}(\vec{r}_p) = \int \hat{I}(\vec{k}, \vec{r}_p) \vec{k} d^2\vec{k} \quad ,
$$

where $\hat{I}(\vec{k})$ is a diffraction pattern intensity. Doing this for every diffraction pattern, we obtain the image shown below. The center of mass is returned as complex `Images`, where the real and imaginary parts correspond to the $x$- and $y$-direction, respectively. We set `units="1/Å"`, hence each complex component is in units of $\mathrm{Å}^{-1}$.

In [ ]:
center_of_mass = measurement_4d.center_of_mass(units="1/Å")

We interpolate and tile, and then show each component.

In [ ]:
plt.close()
%matplotlib inline

interpolated_center_of_mass = center_of_mass.interpolate(0.1).tile((3, 1))

fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4, figsize=(18,4))
interpolated_center_of_mass.real().show(title=r"COM$_x$ (real part)", ax=ax1);
interpolated_center_of_mass.imag().show(title=r"COM$_y$ (imaginary part)", ax=ax2);
interpolated_center_of_mass.intensity().show(title="Amplitude", ax=ax3);
interpolated_center_of_mass.phase().show(title="Phase", ax=ax4);

We can show both real and imaginary components using [domain coloring](https://en.wikipedia.org/wiki/Domain_coloring).

In [ ]:
interpolated_center_of_mass.show(cbar=True, cmap="hsluv", title="COM domain coloring");

It may further be shown, in the weak-phase approximation, that by integrating $\vec{I}_{com}(\vec{r}_p)$, we can obtain the phase change of the exit wave, $\phi(\vec{r_p})$, cross-correlated with the probe intensity

$$
\vec{I}_{iCOM}(\vec{r}_p) = \frac{1}{2\pi} \left[\|\psi_0(\vec{r})\|^2 \star \phi(\vec{r})\right](\vec{r}_p) \quad .
$$

This is the so-called integrated center of mass. We can calculate this using the `integrate_gradient` method, which assumes a complex `Image`.

In [ ]:
integrated_gradient = interpolated_center_of_mass.integrate_gradient()

integrated_gradient.show();

## The frozen phonon model <a id='frozen_phonons'></a>
The atoms in any real material at a particular instance of time are not exactly located at their symmetrical lattice points due to thermal vibrations. In the frozen phonon approximation, the effect of thermal vibrations are simulated by the _intensities_ averaged over several different configurations of atoms with different random offsets. 

To simulate frozen phonons in *ab*TEM, the `Atoms` are wrapped with a `FrozenPhonons` object, where we also need to provide the magnitude of the thermal vibrations for each atomic species and the number of configurations we average over. Including more configurations will be more accurate, but of course also more expensive to calculate.

Getting the right magnitude of thermal vibrations for a particular material is not always trivial, so here we just use the same reasonable value of $0.1 \ \mathrm{Å}$ for all the atomic numbers; note that we could specify this separately for each element. We set the number of random structures in the thermal ensemble to 8.

In [ ]:
atoms = ase.io.read("sto_lto.cif")
abtem.show_atoms(atoms);

In [ ]:
sigmas = {"O": .15, "Sr": .1, "La":.05, "Ti":.1}

frozen_phonons = abtem.FrozenPhonons(atoms * (1, 1, 4), sigmas=sigmas, num_configs=8)

We can draw a particular frozen phonon configuration by iterating. To make the displacements easier to see, we scale down the size of the spheres representing the atoms in the visualization.

In [ ]:
config = next(iter(frozen_phonons))

abtem.show_atoms(config, scale=0.5);

The potential can be created as above, we just provide the frozen phonons instead of the atoms. 

In [ ]:
frozen_phonon_potential = abtem.Potential(
    frozen_phonons, sampling=0.05, slice_thickness=2
)

The potential now has an additional ensemble axis corresponding to the 8 frozen-phonon images.

In [ ]:
print(frozen_phonon_potential.ensemble_axes_metadata)
frozen_phonon_potential.build().array

The potential object can be used in the same way as above, here we do a CBED simulation for each thermal snapshot. The result is an ensemble of 8 wave functions.

In [ ]:
exit_waves = probe.multislice(potential=frozen_phonon_potential)

exit_waves.array

In [ ]:
exit_waves.compute()

The object now has an additional first ensemble axis corresponding to the frozen-phonon image.

In [ ]:
print(exit_waves.ensemble_axes_metadata)
exit_waves.diffraction_patterns().shape

We show the ensemble of wave functions interactively.

In [ ]:
%matplotlib ipympl
exit_waves.diffraction_patterns().show(interact=True);

To get final diffraction pattern, we take the mean over the ensemble dimension.

__Note__: some imaging modes will average over frozen phonons by default to conserve memory.

In [ ]:
mean_diffraction_pattern = exit_waves.diffraction_patterns().mean(0)
mean_diffraction_pattern2 = exit_waves.diffraction_patterns()[::8].mean(0)

We show the resulting diffraction pattern on a power scale.

In [ ]:
plt.close()
%matplotlib inline

(mean_diffraction_pattern - mean_diffraction_pattern2).show(power=0.25, units="mrad", cbar=True);

## Parallalization

*ab*TEM has advanced parallelization capabilities both on CPUs and GPUs, and support for running on compute clusters using MPI.

Please see our documentation on [parallelization](https://abtem.github.io/doc/user_guide/walkthrough/parallelization.html).